# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
###################################################################################
##
##  Python API Homework - What's the Weather Like
##        Part II - VacationPY
##
##
##          Author: George Alonzo
##        Due Date: October 9, 2021
##
###################################################################################

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
#from api_keys import g_key

# Google developer API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Input File (CSV)
input_data_file = "output_data/cities.csv"

city_data = pd.read_csv(input_data_file)
city_data.dropna(inplace = True) 
print(f"Number of records in the input data file: {len(city_data)}")

Number of records in the input data file: 588


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations

coordinates = city_data[["lat", "long"]]

humidity = city_data["humidity"].astype(float)

In [5]:
fig = gmaps.figure(center=(0.0,0.0), zoom_level=1.4)
# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=95,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_cities = city_data.loc[(city_data["maxtemp"]> 70) & 
                             (city_data["maxtemp"]< 80) & 
                             (city_data["windspeed"]< 10) &
                             (city_data["cloudiness"] == 0)]

print(f"Number of cities meeting criteria: {len(ideal_cities)}")

Number of cities meeting criteria: 10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

geo_params = {"key": g_key}

In [8]:
hotel_df = pd.DataFrame(columns =['Hotel Name','City','Country','Lat','Lng'])

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# Use the lat/lng we recovered from ideal cities
for index, row in ideal_cities.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    geo_params["latlng"] = f"{lat},{lng}"

    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request  
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Create URL specific for the Geocode API (to try and obtain city/country for the hotel)
    #    since the hotel may not actually be in the ideal city
    geo_url = "https://maps.googleapis.com/maps/api/geocode/json"  
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = name_address["results"][0]["geometry"]["location"]["lng"]

        # Attempt to obtain city & country for the Lat & Lng via Google Geocode
        #     since hotel may not actually be in the ideal city
        geo_url = (f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={g_key}")
        city_country = requests.get(geo_url)
        city_country = city_country.json()
        try:
            hotel_df.loc[index, "City"] = city_country["results"][0]["address_components"][2]["long_name"]
        except:
            hotel_df.loc[index, "City"] = ""
            print("Unable to locate City for Lat/Lng.")
        try:
            hotel_df.loc[index, "Country"] = city_country["results"][0]["address_components"][len(city_country["results"][0]["address_components"])-2]["long_name"]
        except:
            hotel_df.loc[index, "Country"] = ""
            print("Unable to locate Country for Lat/Lng.")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_df

Missing field/result... skipping.


,Hotel Name,City,Country,Lat,Lng
69,صالون زيرو,Heet,Al Anbar Governorate,33.644258,42.815978
120,Florence Hotel Assiut,Assiut Governorate,Assiut Governorate,27.187037,31.193933
182,Hôtel La Giralda,Nador Province,Oriental,35.1722,-2.9272
303,Austral Hotel,Comodoro Rivadavia,Argentina,-45.86268,-67.476643
381,Goto Guesthouse Utojuku & SA Car Rental,Goto,Japan,32.694457,128.837452
431,"Motel 6 Barstow, CA",Barstow,United States,34.897604,-117.007178
436,Almadina Hotel,Al Wahat District,Al Wahat District,30.757527,20.224074
489,"The Ritz-Carlton, Half Moon Bay",Half Moon Bay,94019,37.433789,-122.441509
551,Hotel Patagonia,Belgrano Sudeste,Río Negro,-41.131426,-71.324583


In [10]:
# Add marker layer ontop of heat map
#fig = gmaps.figure(center=(0.0,0.0), zoom_level=1.4)
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))